# 直接使用问答对（只有国家政策）

In [1]:
import pandas as pd

In [3]:
easy_qa_df = pd.read_excel("D:\Work\讯飞\data\问答对\简单问题汇总_已写入.xlsx", usecols=["问题", "答案", "来源"])
medi_qa_df = pd.read_excel("D:\Work\讯飞\data\问答对\中等问题汇总_已写入.xlsx", usecols=["问题", "答案", "来源"])
hard_qa_df = pd.read_excel("D:\Work\讯飞\data\问答对\复杂问题汇总_已写入.xlsx", usecols=["问题", "答案", "来源"])
real_qa_df = pd.read_excel("D:\Work\讯飞\data\问答对\真实问题汇总_已写入.xlsx", usecols=["问题", "答案", "来源"])

In [7]:
easy_qa_df.shape[0]

441

In [18]:
import numpy as np

easy_idx = np.random.randint(0, easy_qa_df.shape[0], easy_qa_df.shape[0]//4)
medi_idx = np.random.randint(0, medi_qa_df.shape[0], medi_qa_df.shape[0]//4)
hard_idx = np.random.randint(0, hard_qa_df.shape[0], hard_qa_df.shape[0]//4)
real_idx = np.random.randint(0, real_qa_df.shape[0], real_qa_df.shape[0]//4)

In [20]:
easy_res = easy_qa_df.iloc[easy_idx, :]
medi_res = medi_qa_df.iloc[medi_idx, :]
hard_res = hard_qa_df.iloc[hard_idx, :]
real_res = real_qa_df.iloc[real_idx, :]

In [23]:
df = pd.concat([easy_res, medi_res, hard_res, real_res])

In [24]:
df.shape

(204, 3)

In [27]:
df.to_excel("D:\Work\讯飞\data\问答对\召回率测试问题.xlsx", index=False)

# 大模型生成问题

## Connecte Milvus
Get all chunks' id and text from three collections.



In [2]:
from pymilvus import connections, Collection

connections.connect("default", host="localhost", port="19530")

col_country = Collection("CountryPolicy")
col_shanghai = Collection("ShanghaiPolicy")
col_centre = Collection("CentrePolicy")
col_country.load()
col_shanghai.load()
col_centre.load()

In [3]:
output_fields = ["para_id", "text"]
iter_country = col_country.query_iterator(expr='', output_fields=output_fields)
iter_shanghai = col_shanghai.query_iterator(expr='', output_fields=output_fields)
iter_centre = col_centre.query_iterator(expr='', output_fields=output_fields)

# chunk_country = iter_country.next()
# chunk_shanghai = iter_shanghai.next()
# chunk_centre = iter_centre.next()

In [4]:
chunk_country = []
chunk_shanghai = []
chunk_centre = []
while True:
    res_country = iter_country.next()
    res_shanghai = iter_shanghai.next()
    res_centre = iter_centre.next()
    if not res_country:
        iter_country.close()
        iter_shanghai.close()
        iter_centre.close()
        break
    chunk_country += res_country
    chunk_shanghai += res_shanghai
    chunk_centre += res_centre
print(len(chunk_country), len(chunk_shanghai), len(chunk_centre))

6277 2315 227


In [5]:
chunk_country

[{'para_id': '8062649_T01',
  'text': '前言 中华人民共和国城市房地产管理法发布时间：2024-10-23 16:57信息来源：浏览次数：中华人民共和国城市房地产管理法（1994年7月5日第八届全国人民代表大会常务委员会第八次会议通过\u3000根据2007年8月30日第十届全国人民代表大会常务委员会第二十九次会议《关于修改〈中华人民共和国城市房地产管理法〉的决定》第一次修正\u3000根据2009年8月27日第十一届全国人民代表大会常务委员会第十次会议《关于修改部分法律的决定》第二次修正\u3000根据2019年8月26日第十三届全国人民代表大会常务委员会第十二次会议《关于修改〈中华人民共和国土地管理法〉、〈中华人民共和国城市房地产管理法〉的决定》第三次修正）目 录第一章\u3000总 则第二章\u3000房地产开发用地'},
 {'para_id': '8062649_T02', 'text': '第一节 第一节\u3000土地使用权出让'},
 {'para_id': '8062649_T03',
  'text': '第二节 第二节\u3000土地使用权划拨第三章\u3000房地产开发第四章\u3000房地产交易'},
 {'para_id': '8062649_T04', 'text': '第一节 第一节\u3000一般规定'},
 {'para_id': '8062649_T05', 'text': '第二节 第二节\u3000房地产转让'},
 {'para_id': '8062649_T06', 'text': '第三节 第三节\u3000房地产抵押'},
 {'para_id': '8062649_T07', 'text': '第四节 第四节\u3000房屋租赁'},
 {'para_id': '8062649_T08',
  'text': '第五节 第五节\u3000中介服务机构第五章\u3000房地产权属登记管理第六章\u3000法律责任第七章\u3000附 则第一章\u3000总 则'},
 {'para_id': '8062649_T09',
  'text': '第一条 第一条为了加强对城市房地产的管理，维护房地产市场秩序，保障房地产权利人的合法权益，促进房地产业的健康发展，制定本法。'},
 {'

In [6]:
col_country.release()
col_shanghai.release()
col_centre.release()

## Randommly pick one or some chunks as ground truth chunks.
Record the para_id of chosen chunks



In [102]:
import random

# def sample_paragraphs(doc_blocks, min_num=1, max_num=1):
#     num = random.randint(min_num, max_num)
#     sample = random.sample(doc_blocks, num)
#     combined_text = "\n".join([p["text"] for p in sample])
#     para_ids = {p["para_id"] for p in sample}
#     return combined_text, para_ids
import random
import re

def sample_neighbor_paragraphs(doc_blocks, window_size=1):
    # Step 1: 构建 para_id -> index 映射（为了定位）
    pid_to_index = {p["para_id"]: i for i, p in enumerate(doc_blocks)}

    # Step 2: 随机选一个段落
    target_idx = random.randint(0, len(doc_blocks) - 1)
    target_para = doc_blocks[target_idx]
    target_prefix = target_para["para_id"].split("_")[0]  # 提取文档编号前缀
    neighbors = []

    # Step 3: 找到与之相同文档编号的段落索引集合
    same_doc_indices = [i for i, p in enumerate(doc_blocks) if p["para_id"].startswith(target_prefix)]

    # Step 4: 在 same_doc_indices 中找到 target_idx 对应的位置
    if target_idx not in same_doc_indices:
        return "", set()

    relative_pos = same_doc_indices.index(target_idx)
    start = max(0, relative_pos - window_size)
    end = min(len(same_doc_indices), relative_pos + window_size + 1)

    # Step 5: 获取相邻段落（新增过滤）
    selected_indices = same_doc_indices[start:end]
    # selected = [doc_blocks[i] for i in selected_indices]
    selected = [
        doc_blocks[i]
        for i in selected_indices
        if not doc_blocks[i]["text"].strip().startswith("前言")
    ]


    # Step 6: 拼接文本
    combined_text = "\n".join([p["text"] for p in selected])
    para_ids = {p["para_id"] for p in selected}
    return combined_text, para_ids


In [103]:
context_text, para_ids = sample_neighbor_paragraphs(chunk_centre)

In [104]:
context_text

'第十五条 第十五条异议申请主体范围的；（三）异议申请材料不符合格式要求或不齐全的；（四）异议申请未按规定路径提交的；（五）异议申请材料与项目无关或无效的；（六）经营主体拒不配合市交易中心对异议事项进行核查的；（七）异议申请重复提交的；（八）市交易中心给出异议处理结果后或经营主体要求撤回异议后，以同一事项、同一理由再次提出异议的；（九）一年内提出三次及以上异议均查无实据的；（十）其他不予受理的情形。\n第十七条 第十七条（异议答复）市交易中心应当自收到异议申请之日起20 个工作日内，将异议处理结果通过总门户网站书面回复给异议申请主体。异议申请主体可以登录总门户网站，在“投诉处理”中“信用异议-处理结果”板块查看异议处理结果。异议处理期间，不影响异议信息的公开。\n第十八条 第十八条（信息更正）异议成立的，市交易中心应当在异议答复后三个工作日内进行纠正并对经营主体进行说明。'

In [105]:
para_ids

{'8169999_T19', '8169999_T20', '8169999_T21'}

In [ ]:
import numpy as np

chunk_size = np.random.randint(2, 4)
idx = np.random.randint(0, len(chunk_country), 100)
single_chunk_country = 
mult_chunk_country = 
np.random.randint(0. len(chunk_country))
for i in range(chunk_size):
    docs = "\n".join

## Use GLM-4 to generate questions base on chunk(s). 


In [56]:
from langchain_community.chat_models import ChatZhipuAI
from dotenv import load_dotenv

load_dotenv()
model = ChatZhipuAI(model="glm-4-air-250414", temperature=0.5)

def prompt_model_to_generate_question(context_text, para_ids):
#     prompt = f"""请根据以下政策内容生成一个具体的问题，问题应当针对文本的主要内容、要求或执行细节。

# 【政策内容】：
# {context_text}

# 【问题】：
# """
    prompt = f"""请根据以下政策内容生成一个具体的问题，问题应当针对文本的主要内容、要求或执行细节。
注意：
    1、提供的编号分别对应一行政策。
    2、问题尽可能包含每条政策，如果有些政策内容不合适或无法生成问题，请单独告诉我政策编号，并用剩下的政策（如有）生成问题
    3、如果提供的多条政策之间没有关联，请选择一个或相关联的几条提问，并告诉我不合适的编号。
    4、直接输出结果（问题，不合适的编号等），不需要阐述理由，不要输出思考或分析过程，不要回答你生成的问题
    5、不要在问题中提到政策编号，不要说“根据政策第X条...”或“根据政策编号【XXX】的内容...”
    
【政策编号】：
{para_ids}
【政策内容】：
{context_text}

【问题】：
"""
    # 伪代码：请替换成你实际的模型接口
    response = model.invoke(prompt).content
    return response.strip()

## Generate gold entries

In [87]:
def generate_gold_entry(doc_blocks):
    context_text, para_ids = sample_neighbor_paragraphs(doc_blocks)
    question = prompt_model_to_generate_question(context_text, para_ids)
    return {
        "text": context_text,
        "question": question,
        "relevant_para_ids": list(para_ids)
    }

In [106]:
gold_country = [generate_gold_entry(chunk_country) for _ in range(230)]
gold_shanghai = [generate_gold_entry(chunk_shanghai) for _ in range(150)]
gold_centre = [generate_gold_entry(chunk_centre) for _ in range(120)]

In [107]:
gold_country = [{
    "text": gold_country[i].get("text"),
    "question": gold_country[i].get("question").replace('*', '').split("问题：")[-1].replace("\n", "").strip(),
    "relevant_para_ids": gold_country[i].get("relevant_para_ids")} 
                for i in range(len(gold_country))]
# gold_country

In [108]:
gold_shanghai = [{
    "text": gold_shanghai[i].get("text"),
    "question": gold_shanghai[i].get("question").replace('*', '').split("问题：")[-1].replace("\n", "").strip(),
    "relevant_para_ids": gold_shanghai[i].get("relevant_para_ids")} 
                 for i in range(len(gold_shanghai))]
# gold_shanghai

In [109]:
gold_centre = [{
    "text": gold_centre[i].get("text"),
    "question": gold_centre[i].get("question").replace('*', '').split("问题：")[-1].replace("\n", "").strip(),
    "relevant_para_ids": gold_centre[i].get("relevant_para_ids")} 
               for i in range(len(gold_centre))]
# gold_centre

In [110]:
import json

with open("qa_multi_country.json", "w", encoding="utf-8") as f:
    json.dump(gold_country, f, ensure_ascii=False, indent=2)
with open("qa_multi_shanghai.json", "w", encoding="utf-8") as f:
    json.dump(gold_shanghai, f, ensure_ascii=False, indent=2)
with open("qa_multi_centre.json", "w", encoding="utf-8") as f:
    json.dump(gold_centre, f, ensure_ascii=False, indent=2)
with open("qa_multi_all.json", "w", encoding="utf-8") as f:
    json.dump(gold_country+gold_shanghai+gold_centre, f, ensure_ascii=False, indent=2)    

## Concat Milvus Collections

In [30]:
from pymilvus import utility, FieldSchema, CollectionSchema, DataType
connections.connect("default", host="localhost", port="19530")
col_name = "AllPolicy"

fields = [
    FieldSchema(name="para_id", dtype=DataType.VARCHAR, is_primary=True, max_length=100),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8192),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR)
]

schema = CollectionSchema(fields, description="Policy Paragraph Embeddings")


if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

dense_index = {"index_type": "HNSW", "metric_type": "L2"}
col.create_index("dense_vector", dense_index)
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
# col = Collection(col_name)
col.load()

In [31]:
col_country.load()
col_shanghai.load()
col_centre.load()

In [32]:
output_fields = ["para_id", "text", "dense_vector", "sparse_vector"]
iter_country = col_country.query_iterator(expr='', output_fields=output_fields)
iter_shanghai = col_shanghai.query_iterator(expr='', output_fields=output_fields)
iter_centre = col_centre.query_iterator(expr='', output_fields=output_fields)

# chunk_country = iter_country.next()
# chunk_shanghai = iter_shanghai.next()
# chunk_centre = iter_centre.next()

In [33]:
chunk_country = []
chunk_shanghai = []
chunk_centre = []
while True:
    res_country = iter_country.next()
    res_shanghai = iter_shanghai.next()
    res_centre = iter_centre.next()
    if not res_country:
        iter_country.close()
        iter_shanghai.close()
        iter_centre.close()
        break
    chunk_country += res_country
    chunk_shanghai += res_shanghai
    chunk_centre += res_centre
print(len(chunk_country), len(chunk_shanghai), len(chunk_centre))

6277 2315 227


In [35]:
col.insert(chunk_shanghai+chunk_centre)

(insert count: 2542, delete count: 0, upsert count: 0, timestamp: 458468403416399873, success count: 2542, err count: 0

Do question embedding and then store raw question, question vectors, chunks' id into a new collection